<a href="https://colab.research.google.com/github/kedrick07/NLP-Semester-5/blob/main/Lab_5_224232_Kedrick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 5: N-gram Language Processing**
**Course:** SKM3206 – Natural Language Processing  
**Lecturer:** Assoc. Prof. Dr. Azreen Azman / Dr. Nurul Amelina Nasharuddin  
**Due date:** 23 November 2025 (Sunday)

> ⚠️ **Instructions:**  
> 1. Go to *File → Save a copy in Drive* before editing.  
> 2. Rename your notebook as `Lab_LabNo_StudentID_Name.ipynb`.
> 3. Read the examples carefully before attempting the exercises.
> 4. Complete all the tasks in the code cells provided.  
> 5. Submit your Colab link (e.g. http://colab.research.google.com/drive/....) as a submission in the PutraBlast.  





###**💻 Activity**

Review and implement the Python scripts in this [link](https://medium.com/swlh/language-modelling-with-nltk-20eac7e70853) to build an n-gram language model and test it on the next word prediction. Read the 'add-1 smoothing' technique as well. Implement all codes in the code cell below.

In [3]:
import nltk
import re
import string
from nltk import word_tokenize, sent_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords

# Download required NLTK resources
nltk.download('punkt_tab')
nltk.download('stopwords')

# ========================================
# STEP 1: LOAD AND PREPROCESS DATA
# ========================================

# Sample corpus from the Medium article
sample_corpus = """
Alice was beginning to get very tired of sitting by her sister on the bank,
and of having nothing to do: once or twice she had peeped into the book her
sister was reading, but it had no pictures or conversations in it, and what
is the use of a book, thought Alice without pictures or conversations. So she
was considering in her own mind as well as she could, for the hot day made her
feel very sleepy and stupid, whether the pleasure of making a daisy chain would
be worth the trouble of getting up and picking the daisies, when suddenly a
White Rabbit with pink eyes ran close by her. There was nothing so very remarkable
in that; nor did Alice think it so very much out of the way to hear the Rabbit
say to itself, Oh dear! Oh dear! I shall be late! when she thought it over afterwards,
it occurred to her that she ought to have wondered at this, but at the time it all
seemed quite natural; but when the Rabbit actually took a watch out of its waistcoat
pocket, and looked at it, and then hurried on, Alice started to her feet, for it
flashed across her mind that she had never before seen a rabbit with either a
waistcoat pocket, or a watch to take out of it, and burning with curiosity, she
ran across the field after it, and fortunately was just in time to see it pop down
a large rabbit hole under the hedge.
"""

# FIXED: Preprocess properly - preserve spaces
file = sample_corpus.replace("\n", " ")
file = re.sub(r'\s+', ' ', file).strip()  # Normalize whitespace

# Tokenize BEFORE removing punctuation
sents = nltk.sent_tokenize(file)
words = nltk.word_tokenize(file)

print("="*60)
print("CORPUS STATISTICS")
print("="*60)
print(f"The number of sentences is {len(sents)}")
print(f"The number of tokens is {len(words)}")

average_tokens = round(len(words)/len(sents))
print(f"The average number of tokens per sentence is {average_tokens}")

unique_tokens = set([w.lower() for w in words if w.isalnum()])
print(f"The number of unique tokens are {len(unique_tokens)}")
print("="*60)

# ========================================
# STEP 2: BUILD N-GRAM MODELS
# ========================================

print("\nBUILDING N-GRAM MODELS...")
print("="*60)

stop_words = set(stopwords.words('english'))

unigram = []
bigram = []
trigram = []
fourgram = []
tokenized_text = []

for sentence in sents:
    sentence = sentence.lower()
    sequence = word_tokenize(sentence)
    # Remove punctuation but keep words
    sequence = [word for word in sequence if word.isalnum()]

    unigram.extend(sequence)
    tokenized_text.append(sequence)
    bigram.extend(list(ngrams(sequence, 2)))
    trigram.extend(list(ngrams(sequence, 3)))
    fourgram.extend(list(ngrams(sequence, 4)))

# Function to remove ngrams containing only stopwords
def removal(x):
    y = []
    for pair in x:
        count = 0
        for word in pair:
            if word in stop_words:
                count = count or 0
            else:
                count = count or 1
        if count == 1:
            y.append(pair)
    return y

bigram_filtered = removal(bigram)
trigram_filtered = removal(trigram)
fourgram_filtered = removal(fourgram)

freq_bi = nltk.FreqDist(bigram_filtered)
freq_tri = nltk.FreqDist(trigram_filtered)
freq_four = nltk.FreqDist(fourgram_filtered)

print("\nMost common n-grams without stopword removal and without add-1 smoothing:")
print("="*60)
print("Most common bigrams:", freq_bi.most_common(5))
print("\nMost common trigrams:", freq_tri.most_common(5))
print("\nMost common fourgrams:", freq_four.most_common(5))
print("="*60)

# N-grams with stopword removal
print("\n\nMost common n-grams with stopword removal and without add-1 smoothing:")
print("="*60)
unigram_sw_removed = [p for p in unigram if p not in stop_words]
fdist = nltk.FreqDist(unigram_sw_removed)
print("Most common unigrams:", fdist.most_common(10))

bigram_sw_removed = list(ngrams(unigram_sw_removed, 2))
fdist = nltk.FreqDist(bigram_sw_removed)
print("\nMost common bigrams:", fdist.most_common(10))
print("="*60)

# ========================================
# STEP 3: ADD-1 SMOOTHING
# ========================================

print("\n\nAPPLYING ADD-1 SMOOTHING...")
print("="*60)

# Create n-grams for all orders
ngrams_all = {1: [], 2: [], 3: [], 4: []}
for i in range(4):
    for each in tokenized_text:
        for j in ngrams(each, i+1):
            ngrams_all[i+1].append(j)

# Get vocabulary for each n-gram order
ngrams_voc = {1: set([]), 2: set([]), 3: set([]), 4: set([])}
for i in range(4):
    for gram in ngrams_all[i+1]:
        if gram not in ngrams_voc[i+1]:
            ngrams_voc[i+1].add(gram)

# Count total n-grams and vocabulary size
total_ngrams = {1: 0, 2: 0, 3: 0, 4: 0}
total_voc = {1: 0, 2: 0, 3: 0, 4: 0}
for i in range(4):
    total_ngrams[i+1] = len(ngrams_all[i+1])
    total_voc[i+1] = len(ngrams_voc[i+1])

# Calculate smoothed probabilities
ngrams_prob = {1: [], 2: [], 3: [], 4: []}
for i in range(4):
    for ngram in ngrams_voc[i+1]:
        tlist = [ngram]
        tlist.append(ngrams_all[i+1].count(ngram))
        ngrams_prob[i+1].append(tlist)

# Apply Add-1 smoothing
for i in range(4):
    for ngram in ngrams_prob[i+1]:
        ngram[-1] = (ngram[-1] + 1) / (total_ngrams[i+1] + total_voc[i+1])

# Sort and display
print("\nMost common n-grams without stopword removal and with add-1 smoothing:")
print("="*60)
for i in range(4):
    ngrams_prob[i+1] = sorted(ngrams_prob[i+1], key=lambda x: x[1], reverse=True)

print("Most common unigrams:", str(ngrams_prob[1][:10]))
print("\nMost common bigrams:", str(ngrams_prob[2][:10]))
print("\nMost common trigrams:", str(ngrams_prob[3][:10]))
print("\nMost common fourgrams:", str(ngrams_prob[4][:10]))
print("="*60)

# ========================================
# STEP 4: NEXT WORD PREDICTION
# ========================================

print("\n\nNEXT WORD PREDICTION")
print("="*60)

# Test strings
str1 = 'alice was very'
str2 = 'she ran across the'

# Tokenize and clean test strings
token_1 = [w.lower() for w in word_tokenize(str1) if w.isalnum()]
token_2 = [w.lower() for w in word_tokenize(str2) if w.isalnum()]

# Get n-grams from test strings
ngram_1 = {1: [], 2: [], 3: []}
ngram_2 = {1: [], 2: [], 3: []}

for i in range(3):
    ngram_1[i+1] = tuple(token_1[-(i+1):])
    ngram_2[i+1] = tuple(token_2[-(i+1):])

print(f"String 1 n-grams: {ngram_1}")
print(f"String 2 n-grams: {ngram_2}\n")

# Predict next words for string 1
pred_1 = {1: [], 2: [], 3: []}
for i in range(3):
    count = 0
    for each in ngrams_prob[i+2]:
        if each[0][:-1] == ngram_1[i+1]:
            count += 1
            pred_1[i+1].append(each[0][-1])
            if count == 5:
                break
    if count < 5:
        while count != 5:
            pred_1[i+1].append("NOT FOUND")
            count += 1

# Predict next words for string 2
pred_2 = {1: [], 2: [], 3: []}
for i in range(3):
    count = 0
    for each in ngrams_prob[i+2]:
        if each[0][:-1] == ngram_2[i+1]:
            count += 1
            pred_2[i+1].append(each[0][-1])
            if count == 5:
                break
    if count < 5:
        while count != 5:
            pred_2[i+1].append("NOT FOUND")
            count += 1

# Display predictions
print(f"String 1: '{str1}'")
print("="*60)
print(f"Bigram predictions: {pred_1[1]}")
print(f"Trigram predictions: {pred_1[2]}")
print(f"Fourgram predictions: {pred_1[3]}")

print(f"\nString 2: '{str2}'")
print("="*60)
print(f"Bigram predictions: {pred_2[1]}")
print(f"Trigram predictions: {pred_2[2]}")
print(f"Fourgram predictions: {pred_2[3]}")
print("="*60)

print("\n✅ N-gram language model implementation complete!")


CORPUS STATISTICS
The number of sentences is 6
The number of tokens is 284
The average number of tokens per sentence is 47
The number of unique tokens are 137

BUILDING N-GRAM MODELS...

Most common n-grams without stopword removal and without add-1 smoothing:
Most common bigrams: [(('her', 'sister'), 2), (('pictures', 'or'), 2), (('or', 'conversations'), 2), (('rabbit', 'with'), 2), (('the', 'rabbit'), 2)]

Most common trigrams: [(('pictures', 'or', 'conversations'), 2), (('alice', 'was', 'beginning'), 1), (('was', 'beginning', 'to'), 1), (('beginning', 'to', 'get'), 1), (('to', 'get', 'very'), 1)]

Most common fourgrams: [(('alice', 'was', 'beginning', 'to'), 1), (('was', 'beginning', 'to', 'get'), 1), (('beginning', 'to', 'get', 'very'), 1), (('to', 'get', 'very', 'tired'), 1), (('get', 'very', 'tired', 'of'), 1)]


Most common n-grams with stopword removal and without add-1 smoothing:
Most common unigrams: [('rabbit', 5), ('alice', 4), ('sister', 2), ('nothing', 2), ('book', 2), ('

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


---

# 🧮 **Lab Assignment**

### **Instructions:**

1. Modify the codes above to use WSJ corpus as the training  corpus for the n-gram language model.  The WSJ corpus can be can be downloaded from here: https://drive.google.com/file/d/1JMdeLEMC6Zi1XONuDUIYBPJi2M53mcUp/view?usp=drive_link

2. By using *at least 5 incomplete phrases*, evaluate the model for next word prediction.


In [1]:
import nltk
import re
import string
from nltk import word_tokenize, sent_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from collections import Counter

# Download required NLTK resources
nltk.download('punkt_tab')
nltk.download('stopwords')

# ========================================
# STEP 1: CLONE REPO AND LOAD WSJ CORPUS
# ========================================

print("="*60)
print("CLONING GITHUB REPOSITORY...")
print("="*60)

!git clone https://github.com/kedrick07/NLP-Semester-5.git

%cd NLP-Semester-5

print("\n✅ Repository cloned successfully!")
print("="*60)

# Read WSJ corpus from your repo
with open('WSJ Text.txt', 'r', encoding='utf-8') as file:
    corpus = file.read()

print("\n✅ WSJ Corpus loaded from GitHub!")


# ========================================
# STEP 2: PREPROCESS WSJ CORPUS
# ========================================

# Clean text but preserve spaces
corpus = corpus.replace("\n", " ").replace("\r", " ")
corpus = re.sub(r'\s+', ' ', corpus)  # Remove extra whitespace

print("\nCORPUS STATISTICS")
print("="*60)

# Tokenize into sentences first
sents = nltk.sent_tokenize(corpus)
print(f"Number of sentences: {len(sents)}")

# Tokenize into words
words = nltk.word_tokenize(corpus)
print(f"Number of tokens: {len(words)}")

average_tokens = round(len(words)/len(sents)) if len(sents) > 0 else 0
print(f"Average tokens per sentence: {average_tokens}")

unique_tokens = set(words)
print(f"Number of unique tokens: {len(unique_tokens)}")
print("="*60)


# ========================================
# STEP 3: BUILD N-GRAM MODELS
# ========================================

print("\nBUILDING N-GRAM MODELS...")
print("="*60)

stop_words = set(stopwords.words('english'))

tokenized_text = []

for sentence in sents:
    sentence = sentence.lower()
    sequence = word_tokenize(sentence)
    # Remove only punctuation, keep words
    sequence = [word for word in sequence if word.isalnum()]
    if len(sequence) > 0:
        tokenized_text.append(sequence)

# Build n-grams
ngrams_all = {1: [], 2: [], 3: [], 4: []}
for sentence in tokenized_text:
    for i in range(4):
        ngrams_all[i+1].extend(list(ngrams(sentence, i+1)))

print(f"Total unigrams: {len(ngrams_all[1])}")
print(f"Total bigrams: {len(ngrams_all[2])}")
print(f"Total trigrams: {len(ngrams_all[3])}")
print(f"Total fourgrams: {len(ngrams_all[4])}")
print("="*60)


# ========================================
# STEP 4: APPLY ADD-1 SMOOTHING
# ========================================

print("\nAPPLYING ADD-1 SMOOTHING...")
print("="*60)

# Get vocabulary for each n-gram order
ngrams_voc = {1: set([]), 2: set([]), 3: set([]), 4: set([])}
for i in range(4):
    for gram in ngrams_all[i+1]:
        ngrams_voc[i+1].add(gram)

# Count total n-grams and vocabulary size
total_ngrams = {1: 0, 2: 0, 3: 0, 4: 0}
total_voc = {1: 0, 2: 0, 3: 0, 4: 0}
for i in range(4):
    total_ngrams[i+1] = len(ngrams_all[i+1])
    total_voc[i+1] = len(ngrams_voc[i+1])

print(f"Vocabulary sizes:")
print(f"  Unigram: {total_voc[1]}")
print(f"  Bigram: {total_voc[2]}")
print(f"  Trigram: {total_voc[3]}")
print(f"  Fourgram: {total_voc[4]}")

# Calculate smoothed probabilities
ngrams_prob = {1: {}, 2: {}, 3: {}, 4: {}}
for i in range(4):
    for ngram in ngrams_voc[i+1]:
        count = ngrams_all[i+1].count(ngram)
        # Add-1 smoothing
        prob = (count + 1) / (total_ngrams[i+1] + total_voc[i+1])
        ngrams_prob[i+1][ngram] = prob

# Sort by probability
ngrams_prob_sorted = {1: [], 2: [], 3: [], 4: []}
for i in range(4):
    ngrams_prob_sorted[i+1] = sorted(ngrams_prob[i+1].items(), key=lambda x: x[1], reverse=True)

print("✅ Add-1 smoothing applied successfully!")
print("="*60)


# ========================================
# STEP 5: NEXT WORD PREDICTION (5 TEST PHRASES)
# ========================================

print("\n\nNEXT WORD PREDICTION - EVALUATION")
print("="*60)

test_phrases = [
    'the company said',
    'stock market is',
    'according to the',
    'new york times',
    'in the first'
]

def predict_next_words(phrase, ngrams_prob, top_n=5):
    tokens = word_tokenize(phrase.lower())
    tokens = [word for word in tokens if word.isalnum()]

    predictions = {1: [], 2: [], 3: []}

    # Bigram prediction (last 1 word)
    if len(tokens) >= 1:
        context = (tokens[-1],)
        for ngram, prob in ngrams_prob[2]:
            if ngram[:-1] == context:
                predictions[1].append((ngram[-1], prob))
                if len(predictions[1]) >= top_n:
                    break

    # Trigram prediction (last 2 words)
    if len(tokens) >= 2:
        context = tuple(tokens[-2:])
        for ngram, prob in ngrams_prob[3]:
            if ngram[:-1] == context:
                predictions[2].append((ngram[-1], prob))
                if len(predictions[2]) >= top_n:
                    break

    # Fourgram prediction (last 3 words)
    if len(tokens) >= 3:
        context = tuple(tokens[-3:])
        for ngram, prob in ngrams_prob[4]:
            if ngram[:-1] == context:
                predictions[3].append((ngram[-1], prob))
                if len(predictions[3]) >= top_n:
                    break

    # Fill with "NOT FOUND" if needed
    for i in range(1, 4):
        while len(predictions[i]) < top_n:
            predictions[i].append(("NOT FOUND", 0.0))

    return predictions

# Test each phrase
for idx, phrase in enumerate(test_phrases, 1):
    print(f"\n{idx}. Test Phrase: '{phrase}'")
    print("-" * 60)

    predictions = predict_next_words(phrase, ngrams_prob_sorted, top_n=5)

    print(f"Bigram predictions (top 5):")
    for word, prob in predictions[1]:
        print(f"  - {word:15} (probability: {prob:.8f})")

    print(f"\nTrigram predictions (top 5):")
    for word, prob in predictions[2]:
        print(f"  - {word:15} (probability: {prob:.8f})")

    print(f"\nFourgram predictions (top 5):")
    for word, prob in predictions[3]:
        print(f"  - {word:15} (probability: {prob:.8f})")

    print("=" * 60)

print("\n✅ N-gram language model evaluation complete!")
print("="*60)


# ========================================
# DISPLAY MODEL SUMMARY
# ========================================

print("\n\nMODEL STATISTICS SUMMARY")
print("="*60)
print(f"Training corpus: WSJ Text.txt (from GitHub)")
print(f"Total sentences: {len(sents)}")
print(f"Total tokens: {len(words)}")
print(f"Unique tokens: {len(unique_tokens)}")
print(f"N-gram model orders: Unigram, Bigram, Trigram, Fourgram")
print(f"Smoothing technique: Add-1 (Laplace) Smoothing")
print(f"Test phrases evaluated: {len(test_phrases)}")
print("="*60)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


CLONING GITHUB REPOSITORY...
Cloning into 'NLP-Semester-5'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 91 (delta 43), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (91/91), 487.59 KiB | 4.83 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/NLP-Semester-5

✅ Repository cloned successfully!

✅ WSJ Corpus loaded from GitHub!

CORPUS STATISTICS
Number of sentences: 8933
Number of tokens: 212002
Average tokens per sentence: 24
Number of unique tokens: 19109

BUILDING N-GRAM MODELS...
Total unigrams: 173450
Total bigrams: 164547
Total trigrams: 155675
Total fourgrams: 146856

APPLYING ADD-1 SMOOTHING...
Vocabulary sizes:
  Unigram: 14244
  Bigram: 91464
  Trigram: 135629
  Fourgram: 140505
✅ Add-1 smoothing applied successfully!


NEXT WORD PREDICTION - EVALUATION

1. Test Phrase: 'the company said'
--------------------------------------------------------

---
## 📤 **Submission Reminder:**  

Submit the following via PutraBlast:

- The link to your Colab notebook (as *Viewer link*). Ensure the file name is `LabNo_StudentID_Name.ipynb`.
- Output of next word predictions for the 5 strings using the probability models of bigrams, trigrams, and fourgrams, in a text file.
